In [1]:
import requests
import pandas as pd
import io

### Part 1 - Retrieving transaction data

In [2]:
def get_transactions(identifier: str) -> pd.DataFrame:
    '''
    Returns a dataframe with columns 'IDENTIFIER','TIME_PERIOD','OBS_VALUE'
    Parameters:
        identifier(str) : Data Identifier
    return:
        data(dataframe) : Returns a dataframe with column 'IDENTIFIER','TIME_PERIOD','OBS_VALUE'
    '''
    url = 'https://sdw-wsrest.ecb.europa.eu/service/data/BP6/'
    key = identifier+'?detail=dataonly'
    requested_url = url+key
    response = requests.get(requested_url, headers={'Accept': 'text/csv'})
    data = pd.read_csv(io.StringIO(response.text))
    data['IDENTIFIER']=identifier
#     d=data[['IDENTIFIER','TIME_PERIOD','OBS_VALUE']]
    d=data[['IDENTIFIER','TIME_PERIOD','OBS_VALUE']]
#     data=data.replace({'IDENTIFIER':'BP6.',},'',inplace=True)
    return d


In [3]:
try:
    print(get_transactions('Q.N.I8.W1.S1.S1.T.A.FA.D.F._Z.EUR._T._X.N'))
except Exception as ex:
    print(ex)

                                   IDENTIFIER TIME_PERIOD      OBS_VALUE
0   Q.N.I8.W1.S1.S1.T.A.FA.D.F._Z.EUR._T._X.N     1999-Q1   55420.181862
1   Q.N.I8.W1.S1.S1.T.A.FA.D.F._Z.EUR._T._X.N     1999-Q2   87003.970961
2   Q.N.I8.W1.S1.S1.T.A.FA.D.F._Z.EUR._T._X.N     1999-Q3   40331.868800
3   Q.N.I8.W1.S1.S1.T.A.FA.D.F._Z.EUR._T._X.N     1999-Q4  190788.918229
4   Q.N.I8.W1.S1.S1.T.A.FA.D.F._Z.EUR._T._X.N     2000-Q1   97279.053100
..                                        ...         ...            ...
88  Q.N.I8.W1.S1.S1.T.A.FA.D.F._Z.EUR._T._X.N     2021-Q1  127494.694321
89  Q.N.I8.W1.S1.S1.T.A.FA.D.F._Z.EUR._T._X.N     2021-Q2    -485.698879
90  Q.N.I8.W1.S1.S1.T.A.FA.D.F._Z.EUR._T._X.N     2021-Q3   45756.140798
91  Q.N.I8.W1.S1.S1.T.A.FA.D.F._Z.EUR._T._X.N     2021-Q4  -24103.552317
92  Q.N.I8.W1.S1.S1.T.A.FA.D.F._Z.EUR._T._X.N     2022-Q1   66657.274676

[93 rows x 3 columns]


### Part 2 - Retrieving data for the formula

In [92]:
def get_formula_data(formula: str) -> pd.DataFrame:

    '''
    Takes 
    '''
    x=formula.split('=')
    if '+' in x[1]:
        n=x[1].split('+')
    elif '-' in x:
        n=x[1].split('-')
        
    p=get_transactions(n[0])
    q=get_transactions(n[1])
    p.rename(columns={'OBS_VALUE':n[0]}, inplace = True)
    p[n[1]]=q['OBS_VALUE']
    p.drop('IDENTIFIER',axis=1,inplace =True)
    p.set_index('TIME_PERIOD',inplace=True)
    return p

In [93]:
try:   
    print(get_formula_data('Q.N.I8.W1.S1.S1.T.A.FA.D.F._Z.EUR._T._X.N =Q.N.I8.W1.S1P.S1.T.A.FA.D.F._Z.EUR._T._X.N +Q.N.I8.W1.S1Q.S1.T.A.FA.D.F._Z.EUR._T._X.N'))
except Exception as ex:
    print(ex)

             Q.N.I8.W1.S1P.S1.T.A.FA.D.F._Z.EUR._T._X.N  \
TIME_PERIOD                                               
2008-Q1                                   158849.702543   
2008-Q2                                    55317.060368   
2008-Q3                                   136186.079719   
2008-Q4                                    19164.430917   
2009-Q1                                    81747.492243   
2009-Q2                                     4359.016090   
2009-Q3                                    97404.319922   
2009-Q4                                   101643.610182   
2010-Q1                                    88749.530838   
2010-Q2                                   132526.124345   
2010-Q3                                   112994.937630   
2010-Q4                                    16926.193848   
2011-Q1                                   164778.610574   
2011-Q2                                    66837.705827   
2011-Q3                                   101404.079957 

### Part 3 - Computing the aggregates

In [110]:
def compute_aggregates(formula: str) -> pd.DataFrame:
    df=get_transactions(formula)
    
    x=formula.split('=')
    if '+' in x[1]:
        n=x[1].split('+')
    elif '-' in x:
        n=x[1].split('-')
        
    df[x[0]]=df[n[0]]+df[n[1]]
    print(df)

In [112]:
compute_aggregates('Q.N.I8.W1.S1.S1.T.A.FA.D.F._Z.EUR._T._X.N =Q.N.I8.W1.S1P.S1.T.A.FA.D.F._Z.EUR._T._X.N +Q.N.I8.W1.S1Q.S1.T.A.FA.D.F._Z.EUR._T._X.N')